In [1]:
import numpy as np
from experiments.utils.SimpleComparison import SimpleComparison
from hdimvis.algorithms.spring_force_algos.chalmers96_algo.Chalmers96 import Chalmers96
from hdimvis.visualise_layouts_and_metrics.plot import show_layout,show_generation_metrics
from hdimvis.algorithms.stochastic_ntet_algo.SNeD import SNeD
from hdimvis.create_low_d_layout.LayoutCreation import LayoutCreation
from hdimvis.data_fetchers.DataFetcher import DataFetcher
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
from experiments.utils.get_avg_classwise_f1 import get_avg_classwise_f1
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import f1_score
from pathlib import Path
from definitions import PROJECT_ROOT
import pickle
from scipy import stats



In [2]:
output_dir= (Path(PROJECT_ROOT).joinpath(
    Path(f"experiments/sned_vs_96/out/"))).resolve().absolute()

path_to_pickle = (Path(output_dir).joinpath(Path(f"layouts.pickle"))).resolve()
with open(path_to_pickle, 'rb') as file:
    layouts = pickle.load(file)
print(layouts)

{'rna N3k': {'squad': [<hdimvis.create_low_d_layout.SNeDLayout.SNeDLayout object at 0x000001E602973EE0>, <hdimvis.create_low_d_layout.SNeDLayout.SNeDLayout object at 0x000001E602973E50>, <hdimvis.create_low_d_layout.SNeDLayout.SNeDLayout object at 0x000001E64C451C60>, <hdimvis.create_low_d_layout.SNeDLayout.SNeDLayout object at 0x000001E64C4519C0>, <hdimvis.create_low_d_layout.SNeDLayout.SNeDLayout object at 0x000001E64C453070>, <hdimvis.create_low_d_layout.SNeDLayout.SNeDLayout object at 0x000001E6029A4040>, <hdimvis.create_low_d_layout.SNeDLayout.SNeDLayout object at 0x000001E6029A42E0>, <hdimvis.create_low_d_layout.SNeDLayout.SNeDLayout object at 0x000001E6029A4580>, <hdimvis.create_low_d_layout.SNeDLayout.SNeDLayout object at 0x000001E6029A4820>, <hdimvis.create_low_d_layout.SNeDLayout.SNeDLayout object at 0x000001E6029A4AC0>, <hdimvis.create_low_d_layout.SNeDLayout.SNeDLayout object at 0x000001E6029A4D60>, <hdimvis.create_low_d_layout.SNeDLayout.SNeDLayout object at 0x000001E6029A